# 1. Import library 

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, SimpleRNN, LSTM, Dropout, RepeatVector, TimeDistributed, Activation, Flatten, MaxPooling2D
from tensorflow.keras import optimizers

In [2]:
import numpy as np
from scipy.stats import norm
import scipy.optimize as opt
import pandas as pd
import datetime
import time
from arch import arch_model #traditional financial data analysis functions
import matplotlib.pyplot as plt
#from numba import jit
from sklearn.metrics import mean_squared_error as mse
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from numpy import asarray
from pandas import read_csv
import time
sns.set(style='whitegrid', palette='muted')
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 14, 8

In [3]:
X=pd.read_csv("C:/Users/user/Desktop/ECON4305/Data preprocess/x.csv",header=0, index_col=0)
y=pd.read_csv("C:/Users/user/Desktop/ECON4305/Data preprocess/y.csv",header=0, index_col=0)
y=pd.DataFrame(y["Inflation"])
X=X.iloc[:-40]
y=y.iloc[:-40]
y

,Inflation
sasdate,
1992-01-05,0.214977
1992-01-06,0.285919
1992-01-07,0.285104
1992-01-08,0.213295
1992-01-09,0.212842
...,...
2019-01-09,0.176366
2019-01-10,0.285350
2019-01-11,0.217068


# PCA

In [4]:
n=[]
var_explained=[]
for n_comp in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,50,100,120,123]:

    steps_X = [('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
               ('pca', PCA(n_components = n_comp, random_state=1))]
    
    pl_pca_X = Pipeline(steps_X)
    X_pca_model = pl_pca_X.fit(X)
    n.append(n_comp)
    var_explained.append(X_pca_model.steps[1][1].explained_variance_ratio_.sum())
    print('n_components:', n_comp, '  ', 'variance explained:', '%.3f' %X_pca_model.steps[1][1].explained_variance_ratio_.sum())

PCA(n_components = 20, random_state=1)

n_components: 1    variance explained: 0.142
n_components: 2    variance explained: 0.232
n_components: 3    variance explained: 0.310
n_components: 4    variance explained: 0.363
n_components: 5    variance explained: 0.412
n_components: 6    variance explained: 0.449
n_components: 7    variance explained: 0.480
n_components: 8    variance explained: 0.506
n_components: 9    variance explained: 0.530
n_components: 10    variance explained: 0.552
n_components: 11    variance explained: 0.573
n_components: 12    variance explained: 0.592
n_components: 13    variance explained: 0.610
n_components: 14    variance explained: 0.627
n_components: 15    variance explained: 0.644
n_components: 16    variance explained: 0.661
n_components: 17    variance explained: 0.676
n_components: 18    variance explained: 0.691
n_components: 19    variance explained: 0.704
n_components: 20    variance explained: 0.717
n_components: 30    variance explained: 0.821
n_components: 50    variance explained: 0.9

PCA(n_components=20, random_state=1)

In [5]:
steps_X = [('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
           ('pca', PCA(n_components = 40, random_state=1))]

pl_pca_X = Pipeline(steps_X)

X_pca_model = pl_pca_X.fit(X)

print('n_components:', 40, '  ', 'variance explained:', '%.3f' %X_pca_model.steps[1][1].explained_variance_ratio_.sum())

X_pca = X_pca_model.transform(X)

n_components: 40    variance explained: 0.888


In [6]:
X_pca

array([[-0.27247307,  0.63001628, -2.5205931 , ..., -0.48128504,
         0.88484255,  0.42473649],
       [ 0.42807689, -0.27780092, -1.79668044, ..., -0.28923222,
         0.84942813,  0.78773384],
       [ 1.86846141, -0.62097257, -1.51611466, ..., -0.12506208,
         0.60611821,  1.10881978],
       ...,
       [ 2.929683  , -1.70581342,  4.01423412, ..., -0.99449552,
         0.81959715, -0.31924235],
       [-2.65149555,  0.38759858,  0.57019753, ..., -0.82792422,
         1.08534413, -1.40317021],
       [ 0.68455596, -2.14345656,  1.90509682, ..., -1.43620625,
         1.43506522,  0.60505221]])

# Training Model

In [7]:
train_ratio=0.3
n = int(len(X)*train_ratio)
n_steps = 12
n_features = X_pca.shape[1]


In [8]:
# split a multivariate sequence into samples
def split_sequences(X, n_steps):
    X_seq = list()
    for i in range(len(X)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(X):
            break
        # gather input and output parts of the pattern
        seq_x = X[i:end_ix, :]
        X_seq.append(seq_x)
    return np.array(X_seq)

In [9]:
X_train = split_sequences(X_pca[:-n], n_steps)
#X_train = split_sequences(X.iloc[:-1], n_steps)
X_train.shape # no. of instances, no.of timesteps, no.of features

(223, 12, 40)

In [10]:
y_train = y.iloc[n_steps:-n+1].values
#y_train = y.iloc[n_steps:].values
y_train


array([[ 0.27777796],
       [ 0.06932409],
       [ 0.13850418],
       [ 0.20739724],
       [ 0.13802625],
       [ 0.41293933],
       [ 0.27434859],
       [ 0.20526863],
       [ 0.        ],
       [ 0.27303771],
       [ 0.27229425],
       [ 0.06795787],
       [ 0.20359695],
       [ 0.27081939],
       [ 0.3374961 ],
       [ 0.40349752],
       [ 0.20113986],
       [ 0.06695682],
       [ 0.26737984],
       [ 0.20006676],
       [ 0.26613455],
       [ 0.26542816],
       [ 0.1986098 ],
       [ 0.39604012],
       [ 0.19743343],
       [ 0.1970444 ],
       [ 0.13114756],
       [ 0.19639941],
       [ 0.13071897],
       [ 0.26092644],
       [ 0.13020835],
       [ 0.13003903],
       [ 0.51847167],
       [ 0.19373593],
       [ 0.32206147],
       [ 0.38510959],
       [ 0.19200006],
       [ 0.19163213],
       [ 0.1912656 ],
       [ 0.12730746],
       [ 0.3175614 ],
       [ 0.31655614],
       [ 0.31555722],
       [ 0.25173078],
       [ 0.1883831 ],
       [ 0

In [11]:
X_test = split_sequences(X_pca[-n-n_steps+1:], n_steps)
#X_test = split_sequences(X.iloc[-n-n_steps+1:], n_steps)
X_test.shape

(99, 12, 40)

In [19]:
#Define the LSTM model
model = Sequential()
model.add(BatchNormalization())
model.add(LSTM(40, activation='elu', kernel_regularizer="l2", input_shape=(n_steps, n_features)))

model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))
adam = optimizers.Adam(lr=0.005)
model.compile(optimizer=adam, 
              loss='mean_squared_error',
              metrics=['mse'])
np.random.seed(1)
tf.random.set_seed(2)

#Train the LSTM Model
model_lstm = model.fit(X_train,
                       y_train,
                       batch_size=20, epochs=1000, shuffle=False, verbose=False)

#Predict with test data
LSTM_pred = model.predict(X_test)
LSTM_pred.shape
LSTM_train_pred = model.predict(X_train)

rmse_lstm = np.sqrt(mse(y.iloc[-n:], LSTM_pred))
rmse_lstm

#Calculate the RMSE, correlation and R2
LSTM_pred=pd.DataFrame(LSTM_pred,index=y.iloc[-n:].index)
results=pd.concat([y.iloc[-n:],LSTM_pred],axis=1,ignore_index=True)
results.columns=["Actual","LSTM"]
results.index = pd.DatetimeIndex(results.index).strftime("%d-%y")

print("test error: ", metrics.mean_squared_error(y.iloc[-n:], LSTM_pred,squared=False))
print("train error", metrics.mean_squared_error(y.iloc[:-n-n_steps+1], LSTM_train_pred))
print("Correlation: ",results.corr() )
print("R2: ", metrics.r2_score(y.iloc[-n:], LSTM_pred))


#Plot graph 
(results["Actual"]).plot(color="blue")
(results["LSTM"]).plot(color="black")
plt.legend(["Actual","LSTM"])
plt.xlabel("date (mm-yy)")
plt.ylabel("Inflation rate")
plt.grid(True)
plt.show()

7/7 [==============================] - 0s 3ms/step


TypeError: bad operand type for unary -: 'list'